### Recommendations Rank & Knowledge Based: 


In [1]:
import numpy as np
import pandas as pd

In [5]:
# Read the datasets
movies = pd.read_csv('movies_clean.csv', index_col='movie_id')
del movies['Unnamed: 0']

reviews = pd.read_csv('reviews_clean.csv', index_col=0)

In [6]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
reviews.shape

(712337, 23)

In [4]:
movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
reviews[['user_id', 'movie_id']].duplicated().sum()
# No duplicates means that each user has only one review for a movie

0

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [6]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Work on the relevant columns only. 
    df = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
    # Get movies with at least 5 ratings.
    count_df = df.groupby('movie_id').count()
    filtered_movies_idx = count_df[count_df['rating'] > 4].index.to_list()
    # Calculate the average rating for the filtered movies.
    # Sort (by average) Descending order.
    # Run an inner sorting by ratings_count Descending order.
    # Run another innner sorting by the (avg or max) time-of-rating descending.
    result = (df
              .groupby('movie_id')
              .agg({'rating': ['mean', 'count'], 'timestamp': ['max']})
              .loc[filtered_movies_idx, ['rating', 'timestamp']]
              .sort_values(by=[
                  ('rating', 'mean'),
                  ('rating', 'count'),
                  ('timestamp', 'max'),
              ],
               ascending=[False, False, False],
    ))
    top_movies_idx = result.index.to_list()[:n_top]
    top_movies = movies.loc[top_movies_idx, 'movie'].to_list()
    return top_movies # a list of the n_top movies as recommended

In [100]:
popular_recommendations(user_id=1, n_top=3, years=[2014], genres=['History'])

['Birlesen Gonuller (2014)', 'Night Will Fall (2014)', 'Red Army (2014)']

After Implementing the above function go to the practical quiz in maharatech link and solve the MCQ there. 

In [101]:
popular_recommendations(user_id='1', n_top=3, years=[2015, 2016, 2017, 2018], genres=['Romance'])

['Koe no katachi (2016)', 'Departure (2015)', 'Hepta: The Last Lecture (2016)']

In [102]:
popular_recommendations(user_id='1', n_top=100)[-1]

"One Flew Over the Cuckoo's Nest (1975)"

In [103]:
popular_recommendations(user_id='1', n_top=3)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)']

In [104]:
popular_recommendations(user_id='1', n_top=100, genres=['History', 'News'])[-1]

'Elizabeth (1998)'

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

The shape of the function should be as below:
```
popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [7]:
def popular_recommendations(user_id, n_top, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Work on the relevant columns only. 
    df = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
    
    # Get movies with at least 5 ratings.
    count_df = df.groupby('movie_id').count()
    filtered_movies_idx = count_df[count_df['rating'] > 4].index.to_list()
#     print('After filtering [rating > 4], len(filtered_movies_idx) =', len(filtered_movies_idx))
    
    # Filter upon years (if exist).
    _movies = None
    if years:
        if not _movies:
#             print('Creating _movies (years-if-block) ...')
            _movies = movies.loc[filtered_movies_idx, ['movie', 'genre', 'date']]
#             print('movies.shape =', movies.shape)
#             print('_movies.shape =', _movies.shape)
        
        yrs = [int(y) for y in years]
        filtered_movies_idx = _movies[_movies['date'].isin(yrs)].index.to_list()
#         print('After filtering years, len(filtered_movies_idx) =', len(filtered_movies_idx))
        _movies = _movies.loc[filtered_movies_idx]
#         print(list(_movies['date'].unique()))
#         print('After filtering years, _movies.shape =', _movies.shape)
    
    # Filter upon genres (if exist).
    if genres:
        if _movies is None:
#             print('Creating _movies (genres-if-block) ...')
            _movies = movies.loc[filtered_movies_idx, ['movie', 'genre', 'date']]
#             print('movies.shape =', movies.shape)
#             print('_movies.shape =', _movies.shape)
        _movies = _movies.fillna('NONE')
        filtered_movies_idx = _movies[_movies['genre'].str.contains('|'.join(genres))].index.to_list()
#         print('After filtering genres, len(filtered_movies_idx) =', len(filtered_movies_idx))
    # Calculate the average rating for the filtered movies.
    # Sort (by average) Descending order.
    # Run an inner sorting by ratings_count Descending order.
    # Run another innner sorting by the (avg or max) time-of-rating descending.
    result = (df
              .groupby('movie_id')
              .agg({'rating': ['mean', 'count'], 'timestamp': ['max']})
              .loc[filtered_movies_idx, ['rating', 'timestamp']]
              .sort_values(
                  by=[
                      ('rating', 'mean'),
                      ('rating', 'count'),
                      ('timestamp', 'max'),
                  ],
                  ascending=[False, False, False],
    ))
    top_movies_idx = result.head(n_top).index.to_list()
#     result[['movie', 'genre']] = movies.loc[top_movies_idx, ['movie', 'genre']]
#     display(result.head(n_top))
    top_movies = movies.loc[top_movies_idx, 'movie'].to_list()
    return top_movies # a list of the n_top movies as recommended

In [82]:
popular_recommendations('1', 20)

rating         timestamp  \
               mean count         max   
movie_id                                
4921860   10.000000    48  1471195010   
5262972   10.000000    28  1452213883   
5688932   10.000000    14  1529199888   
2737018   10.000000    10  1431298561   
2560840   10.000000     6  1453509044   
2219210   10.000000     6  1374535852   
4448444   10.000000     5  1504965108   
5131914   10.000000     5  1459749142   
2059318   10.000000     5  1392306511   
1431149   10.000000     5  1374859141   
5512872    9.985836   353  1526516958   
4148400    9.882353    17  1445556877   
6798422    9.800000     5  1526423974   
111341     9.800000     5  1520582994   
423176     9.800000     5  1436063266   
1629443    9.800000     5  1394926677   
2592910    9.666667     9  1379684627   
58888      9.666667     6  1503012354   
2396421    9.666667     6  1483505894   
12364      9.625000     8  1475259604   

                                                      movie  \
                                                              
movie_id                                                      
4921860                          MSG 2 the Messenger (2015)   
5262972               Avengers: Age of Ultron Parody (2015)   
5688932                          Sorry to Bother You (2018)   
2737018                                        Selam (2013)   
2560840   Quiet Riot: Well Now You're Here, There's No W...   
2219210                            Crawl Bitch Crawl (2012)   
4448444                              Make Like a Dog (2015)   
5131914                                    Pandorica (2016)   
2059318                                Third Contact (2011)   
1431149                                 Romeo Juliet (2009)   
5512872                                  Be Somebody (2016)   
4148400                            Birlesen Gonuller (2014)   
6798422                                      Agnelli (2017)   
111341                                    Sátántangó (1994)   
423176                                        Shijie (2004)   
1629443                                       Foster (2011)   
2592910                       CM101MMXI Fundamentals (2013)   
58888                                        Akahige (1965)   
2396421   Crystal Lake Memories: The Complete History of...   
12364                                      Körkarlen (1921)   

                            genre  
                                   
movie_id                           
4921860      Comedy|Drama|Fantasy  
5262972              Short|Comedy  
5688932     Comedy|Fantasy|Sci-Fi  
2737018             Drama|Romance  
2560840         Documentary|Music  
2219210    Horror|Sci-Fi|Thriller  
4448444        Short|Comedy|Drama  
5131914                    Sci-Fi  
2059318   Mystery|Sci-Fi|Thriller  
1431149                     Drama  
5512872              Comedy|Drama  
4148400             Drama|History  
6798422               Documentary  
111341               Comedy|Drama  
423176                      Drama  
1629443       Comedy|Drama|Family  
2592910        Documentary|Comedy  
58888                       Drama  
2396421               Documentary  
12364        Drama|Fantasy|Horror

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Sorry to Bother You (2018)',
 'Selam (2013)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)',
 'Make Like a Dog (2015)',
 'Pandorica (2016)',
 'Third Contact (2011)',
 'Romeo Juliet (2009)',
 'Be Somebody (2016)',
 'Birlesen Gonuller (2014)',
 'Agnelli (2017)',
 'Sátántangó (1994)',
 'Shijie (2004)',
 'Foster (2011)',
 'CM101MMXI Fundamentals (2013)',
 'Akahige (1965)',
 'Crystal Lake Memories: The Complete History of Friday the 13th (2013)',
 'Körkarlen (1921)']